In [51]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report

In [52]:
import pandas as pd
data = pd.read_csv("D:\\SchemeAssist-AI\\aid105-Nitesh9842\\data\\combined_schemes.csv")
print(data.head())

  scheme_id                          scheme_name    level state     category  \
0    SCH001                PM Kisan Samman Nidhi  Central   All  Agriculture   
1    SCH002  Pradhan Mantri Awas Yojana (Gramin)  Central   All      Housing   
2    SCH003   Pradhan Mantri Awas Yojana (Urban)  Central   All      Housing   
3    SCH004              Ayushman Bharat – PMJAY  Central   All       Health   
4    SCH005       Pradhan Mantri Jan Dhan Yojana  Central   All      Finance   

   min_age  max_age  min_income  max_income              target_group  \
0       18      100           0      200000  Small & Marginal Farmers   
1       18      100           0      180000                Rural Poor   
2       18      100           0      200000                Urban Poor   
3        0      100           0      500000              EWS Families   
4       18      100           0      999999              All Citizens   

                         benefits is_active last_updated       source details  \

In [53]:
from sklearn.preprocessing import LabelEncoder

# Handle missing values if any
data['min_age'] = data['min_age'].fillna(18)
data['max_age'] = data['max_age'].fillna(65)
data['min_income'] = data['min_income'].fillna(0)
data['max_income'] = data['max_income'].fillna(999999)

# Encode categorical variables
label_encoder_state = LabelEncoder()
label_encoder_category = LabelEncoder()

data['state_encoded'] = label_encoder_state.fit_transform(data['state'])
data['category_encoded'] = label_encoder_category.fit_transform(data['category'])

# Select relevant features for X (using encoded versions)
x = data[['min_age', 'max_age', 'min_income', 'max_income', 'state_encoded', 'category_encoded']]

# Select target variable
y = data['scheme_name']

print(f"Features shape: {x.shape}")
print(f"Number of unique schemes: {y.nunique()}")

Features shape: (3581, 6)
Number of unique schemes: 3581


In [54]:
print (x)

      min_age  max_age  min_income  max_income  state_encoded  \
0          18      100           0      200000              0   
1          18      100           0      180000              0   
2          18      100           0      200000              0   
3           0      100           0      500000              0   
4          18      100           0      999999              0   
...       ...      ...         ...         ...            ...   
3576       18       65           0      999999              0   
3577       18       65           0      999999              2   
3578       18       65           0      999999              2   
3579       18       65           0      999999              2   
3580       18       65           0      999999              2   

      category_encoded  
0                    0  
1                    8  
2                    8  
3                    7  
4                    6  
...                ...  
3576                 3  
3577               

In [55]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

# Initialize and train the model


In [56]:
print(len(x_train), len(y_train))
print(len(x_test), len(y_test))


2864 2864
717 717


In [57]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    n_estimators=100,
    max_depth=20,
    min_samples_split=5,
    random_state=42,
    n_jobs=-1
)

In [58]:
print("Training the model...")
model.fit(x_train, y_train)
print("Model training completed!")

# Evaluate on training set
train_accuracy = model.score(x_train, y_train)
print(f"\nTraining Accuracy: {train_accuracy:.4f}")

# Evaluate on test set
test_accuracy = model.score(x_test, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")

Training the model...


d:\SchemeAssist-AI\.venv\Lib\site-packages\sklearn\ensemble\_forest.py:830: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  check_classification_targets(y)
d:\SchemeAssist-AI\.venv\Lib\site-packages\sklearn\tree\_classes.py:297: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  check_classification_targets(y)
d:\SchemeAssist-AI\.venv\Lib\site-packages\sklearn\tree\_classes.py:297: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  check_classification_targets(y)
d:\SchemeAssist-AI\.venv\Lib\site-packages\sklearn\tree\_classes.py:297: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression pr

Model training completed!

Training Accuracy: 0.0388
Test Accuracy: 0.0000


In [59]:
# Make predictions and get top recommendations
y_pred = model.predict(x_test)

# Display sample predictions
print("\nSample Predictions (first 5):")
for i in range(min(5, len(y_test))):
    print(f"Actual: {y_test.iloc[i]}")
    print(f"Predicted: {y_pred[i]}")
    print("---")


Sample Predictions (first 5):
Actual: Gopal Incentive Scheme
Predicted: Amar Fasal Amar Gola
---
Actual: AlCTE - Distinguished Professional Scheme
Predicted: Post Doctoral Fellowship to Scheduled Caste/Scheduled Tribe Candidates
---
Actual: Coir Vikas Yojana-Skill Upgradation and Mahila Coir Yojana
Predicted: Advance Authorisation(AA)
---
Actual: Scheme to Provide Financial Assistance to Farmers for Purchasing Sanedo Agricultural Equipment
Predicted: Amar Fasal Amar Gola
---
Actual: Housing Assistance (Madhya Pradesh)
Predicted: Loans and Advances for Construction of a House
---


In [60]:
# Save the model for later use
import pickle

with open('scheme_model.pkl', 'wb') as f:
    pickle.dump(model, f)

with open('label_encoder_state.pkl', 'wb') as f:
    pickle.dump(label_encoder_state, f)
    
with open('label_encoder_category.pkl', 'wb') as f:
    pickle.dump(label_encoder_category, f)

print("Model and encoders saved successfully!")

Model and encoders saved successfully!


In [61]:
# Better Approach: Eligibility-Based Recommendation System
import numpy as np

def recommend_schemes(user_age, user_income, user_state, user_category, top_n=10):
    """
    Recommend schemes based on eligibility criteria matching
    """
    # Filter eligible schemes
    eligible_mask = (
        (data['min_age'] <= user_age) & 
        (data['max_age'] >= user_age) &
        (data['min_income'] <= user_income) &
        (data['max_income'] >= user_income) &
        ((data['state'] == user_state) | (data['state'] == 'All'))
    )
    
    if user_category:
        eligible_mask &= ((data['category'] == user_category) | (data['category'].isna()))
    
    eligible_schemes = data[eligible_mask].copy()
    
    if len(eligible_schemes) == 0:
        return pd.DataFrame()
    
    # Calculate relevance score
    eligible_schemes['score'] = 0.0
    
    # State match bonus
    eligible_schemes.loc[eligible_schemes['state'] == user_state, 'score'] += 30
    
    # Category match bonus
    if user_category:
        eligible_schemes.loc[eligible_schemes['category'] == user_category, 'score'] += 30
    
    # Age relevance (prefer schemes with wider age range or closer to user age)
    age_range = eligible_schemes['max_age'] - eligible_schemes['min_age']
    eligible_schemes['score'] += (1 / (age_range + 1)) * 20
    
    # Income relevance (prefer schemes with appropriate income range)
    income_fit = np.minimum(
        (user_income - eligible_schemes['min_income']) / (eligible_schemes['max_income'] - eligible_schemes['min_income'] + 1),
        1.0
    )
    eligible_schemes['score'] += income_fit * 20
    
    # Sort by score
    top_schemes = eligible_schemes.nlargest(top_n, 'score')
    
    return top_schemes[['scheme_name', 'category', 'state', 'benefits', 'score']]

print("Recommendation function created successfully!")

Recommendation function created successfully!


In [62]:
# Test the recommendation system
test_user = {
    'age': 25,
    'income': 100000,
    'state': 'All',
    'category': 'Education'
}

print(f"\nRecommending schemes for:")
print(f"Age: {test_user['age']}, Income: ₹{test_user['income']:,}, State: {test_user['state']}, Category: {test_user['category']}")
print("\n" + "="*100)

recommendations = recommend_schemes(
    test_user['age'], 
    test_user['income'], 
    test_user['state'], 
    test_user['category'],
    top_n=10
)

print(f"\nTop {len(recommendations)} Recommended Schemes:\n")
for idx, row in recommendations.iterrows():
    print(f"🎯 {row['scheme_name']}")
    print(f"   Category: {row['category']} | State: {row['state']} | Score: {row['score']:.2f}")
    print(f"   Benefits: {row['benefits']}")
    print()


Recommending schemes for:
Age: 25, Income: ₹100,000, State: All, Category: Education


Top 10 Recommended Schemes:

🎯 Post Matric Scholarship for OBC Students
   Category: Education | State: All | Score: 81.00
   Benefits: Tuition fee + maintenance

🎯 ONGC Foundation Scholarship
   Category: Education | State: All | Score: 72.22
   Benefits: ₹48000 annually

🎯 LIC Golden Jubilee Scholarship
   Category: Education | State: All | Score: 71.82
   Benefits: ₹10000-20000 annually

🎯 Post Matric Scholarship for Minority Students
   Category: Education | State: All | Score: 71.00
   Benefits: Full tuition + living expenses

🎯 Sitaram Jindal Foundation Scholarship
   Category: Education | State: All | Score: 70.22
   Benefits: ₹500-2000 monthly

🎯 Merit-cum-Means Scholarship for Professional Courses
   Category: Education | State: All | Score: 69.43
   Benefits: Full tuition up to ₹20000

🎯 Post Matric Scholarship for SC Students
   Category: Education | State: All | Score: 69.00
   Benefits: